# Pushes

MAU率(MAU/インストール数)を計算するNotebook

##Input Parameters
+ DATE 集計期間の終わりの日
+ DEBUG 手動実行時のみTrue
+ FREQUENCY 実行頻度
+ BIGQUERY_PROJECT_ID bigqueryのプロジェクト名
+ BIGQUERY_DATASET bigqueryのデータセット
+ PACKAGE_NAME bigqueryのパッケージ名
+ OUTPUT_BIGQUERY_PROJECT_ID 出力先のBQのプロジェクト名
+ ISLATEST 最新の日付を対象にする場合はTrue,任意の日付を指定する場合はFalse

# Output Range
+ daily
DATEの1日前を対象に集計
 
  ex.DATE="2021-02-02"の場合は"2021-02-01を対象に集計"
+ weekly
DATEの1日前から7日を対象に集計

  ex.DATE="2021-02-22"の場合は"2021-02-15"から"2021-02-21を対象に集計"
+ monthly
DATEの1日前から1ヶ月を対象に集計

  ex.DATE="2021-02-01"の場合は"2021-01-01"から"2021-01-31"を対象に集計"

## Output Data
+ MAU
+ Mdownload
+ Mpercent

# Parameters

In [14]:
DATE = "2021-08-17" # @param {type: "date"}
DEBUG = True # @param {type: "boolean"} 手動実行時のみTrueにする。Cloud FunctionsからFalseを渡される。
FREQUENCY = "daily" # @param {type: "string"}
BIGQUERY_PROJECT_ID = "komtar-monet-prd" # @param {type: "string"}
BIGQUERY_DATASET = "analytics_167835138.events_*" # @param {type: "string"}
PACKAGE_NAME = "jp.co.fuller.snowpeak" # @param {type: "string"}
OUTPUT_BIGQUERY_PROJECT_ID = "fl-komtar-analytics-dashboard" # @param {type: "string"}
IS_LATEST = False # @param {type:"boolean"}

# Constants

In [2]:
METRICS_NAME = "pushes"

# Version

In [3]:
VERSION = "1"

# Authorize

In [4]:
if DEBUG:
    from google.colab import auth
    auth.authenticate_user()

# Imports

In [5]:
from datetime import datetime, timedelta
import pandas as pd
from pytz import timezone 

# Get Input Datasets

## 対象範囲の計算

In [16]:
if IS＿LATEST:
  date = (datetime.now(timezone("Asia/Tokyo"))-timedelta(days=1))
else:
  date = datetime.strptime(DATE,"%Y-%m-%d") - timedelta(days=1)
end = date
start = datetime(end.year,end.month,1)
start_date = start.strftime("%Y%m%d")
end_date = end.strftime("%Y%m%d")
start_date, end_date

('20210801', '20210816')

In [17]:
base_start = start-timedelta(days=1)
base_end = end+timedelta(days=1)

## Query

### プッシュ通知の数を取得

In [22]:
query = f"""
SELECT *
FROM `{PACKAGE_NAME.replace(".","_")}_downloads.{FREQUENCY}_events_*`
WHERE _table_suffix BETWEEN "{start_date}" AND "{end_date}"
"""
df_download = pd.read_gbq(query,project_id=OUTPUT_BIGQUERY_PROJECT_ID, dialect='standard')
df_download

,date,appstore_installs,googleplay_installs,installs
0,2021-08-09,741,86,827
1,2021-08-11,488,98,586
2,2021-08-04,352,47,399
3,2021-08-16,570,83,653
4,2021-08-03,360,69,429
5,2021-08-12,630,93,723
6,2021-08-08,669,111,780
7,2021-08-07,595,98,693
8,2021-08-10,517,66,583
9,2021-08-06,344,67,411


In [23]:
query = f"""
SELECT *
FROM `{PACKAGE_NAME.replace(".","_")}_active_users.monthly_events_*`
WHERE _table_suffix BETWEEN "{start_date}" AND "{end_date}"
"""
df_mau = pd.read_gbq(query,project_id=OUTPUT_BIGQUERY_PROJECT_ID, dialect='standard')
df_mau

,date,android_active_users,ios_active_users,total_users
0,2021-08-01,77527,522907,600434


In [28]:
potential = df_mau["total_users"]/df_download["installs"].sum()
potential

0    58.860308
Name: total_users, dtype: float64

In [ ]:
df_output = df_events
df_output.insert(0, "date", start.strftime(format="%Y-%m-%d"))
df_output["date"] = pd.to_datetime(df_output["date"]).dt.date
df_output.to_gbq(f"""{PACKAGE_NAME.replace(".","_")}_{METRICS_NAME.replace("-","_")}.{FREQUENCY}_events_{start.strftime(format="%Y-%m-%d").replace("-","")}""",
                 if_exists="replace",
                 table_schema=[{'name': 'date','type': 'DATE'},
                               {'name': 'android_active_users','type': 'INT64'},
                               {'name': 'ios_active_users','type': 'INT64'},
                               {'name': 'total_users','type': 'INT64'}],
                 project_id=OUTPUT_BIGQUERY_PROJECT_ID)
df_output

1it [00:07,  7.98s/it]


,date,pushs
0,2021-05-04,7
